In [ ]:
""" Makeshift terminal for running script when there is no (free) terminal available """

# 0 = encode  dataset, 1 = train model, 2 = inference image
usage = 2

# Check Specification

In [ ]:
# Check OS

!cat /etc/os-release

In [ ]:
# Check machine

!uname -m

In [ ]:
# Check processor

!cat /proc/cpuinfo

In [ ]:
# Check memory

!cat /proc/meminfo

In [ ]:
# Check env

!env

In [ ]:
# Check GPU

!nvidia-smi

In [ ]:
# Check CUDA

!nvcc --version

In [ ]:
# Check GPU type

!pip install tensorflow

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Setup Workplace

## Install libraries

In [ ]:
# Main libraries

match usage:
    case 0:
        !pip install --upgrade jax jaxlib
        !pip install webdataset
        !pip install git+https://github.com/patil-suraj/vqgan-jax.git
    case 1:
        !pip install datasets
        !pip install datasets[streaming]
        !pip install --upgrade jax jaxlib
        !pip install flax
        !pip install transformers
        !pip install wandb
        !pip install einops
        !pip install emoji
        !pip install ftfy
        !pip install unidecode
        !pip install braceexpand
        !pip install tensorflow
        !pip install dalle-mini
        !pip install optax-shampoo
    case 2:
        !pip install flax==0.6.3 chex==0.1.6 orbax==0.1.2 optax # to resolve orbax update
        !pip install dalle-mini
        !pip install git+https://github.com/patil-suraj/vqgan-jax.git
        !pip install -U jax==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_releases.html
        !pip install -U jaxlib==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_releases.html
        !pip install gradio
        !pip install tensorrt
        !pip install wandb
    case _:
        print("What are you going to do? Please check the usage.")

# Jax and jaxlib according to each CUDA and CuDNN version

import os

match usage:
    case 0 | 1:
        if(os.path.exists("/content")):
            !pip install -U jax==0.4.9 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
            !pip install -U jaxlib==0.4.9+cuda11.cudnn86 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

        elif(os.path.exists("/notebooks")):
            !pip install -U jax==0.4.7 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
            !pip install -U jaxlib==0.4.7+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

        elif(os.path.exists("/kaggle")):
            !pip install -U jax==0.4.7 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
            !pip install -U jaxlib==0.4.7+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

    case 2:
        !pip install -U jax==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
        !pip install -U jaxlib==0.3.25+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

    case _:
        print("What are you going to do? Please check the usage.")

### Check versions

In [ ]:
# Check python and pip version

!python --version
!pip --version

In [ ]:
# Check all libraries installed

!pip list

## Login W&B

In [ ]:
import wandb

if(usage == 1 or usage == 2):
    if(os.path.exists("/content")):
        from google.colab import drive
        drive.mount('/content/drive')

        with open("/content/drive/MyDrive/Colab Notebooks/wandb_auth_key.txt") as wak:
            wandb_auth_key = wak.readline()

    elif(os.path.exists("/kaggle")):
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        wandb_auth_key = user_secrets.get_secret("wandb")

    else:
        # Gradient's secret only limited to premium private clusters
        wandb_auth_key = "Enter_your_WANDB_authentication_key"

    wandb.login(key=wandb_auth_key)

## Get additional files

In [ ]:
# If using Colab, get dataset from Drive for training

if usage == 1
    if os.path.exists("/content"):
        from google.colab import drive
        drive.mount('/content/drive')

        !mkdir "/content/encoded_data"
        !unzip "/content/tugas-akhir/train/datasets/encoded_256_bs60_sf60.zip" -d "/content/encoded_data"

    elif os.path.exists("/notebooks"):
        !mkdir "/notebooks/github/train/datasets/encoded_data"
        !unzip "/notebooks/github/train/datasets/encoded_256_bs60_sf60.zip" -d "/notebooks/encoded_data"

In [ ]:
# Get files from GitHub; only for Kaggle and Colab as Gradient use separated terminal
import sys

username = "fedorajuandy"
reponame = "tugas-akhir"

if(os.path.exists("/content")):
    from google.colab import drive
    drive.mount('/content/drive')

    !rm -r /content/tugas-akhir

    url = f"https://github.com/fedorajuandy/tugas-akhir.git"

    get_ipython().system(f"git clone {url}")

elif(os.path.exists("/kaggle")):
    !rm -r /kaggle/working/tugas-akhir

    url = f"https://github.com/fedorajuandy/tugas-akhir.git"\

    get_ipython().system(f"git clone {url}")
    sys.path.append("tugas-akhir")

else:
    print("Where are you?")

# Run Script

In [ ]:
match usage:
    case 0:
        if(os.path.exists("/content")):
            args = "/content/tugas-akhir/encode/encode_dataset.py"

        elif(os.path.exists("/notebooks")):
            args = "/notebooks/github/encode/encode_dataset.py"

        elif(os.path.exists("/kaggle")):
            args = "/kaggle/working/tugas-akhir/encode/encode_dataset.py"

        else:
            print("Where are you?")

    case 1:
        tokenizer_name = "boris/dalle-mini-tokenizer"
        per_device_train_batch_size = 60
        num_train_epochs = 6
        save_steps = 3000
        mp_devices = 1
        assert_tpu_available = False
        dtype = "float32"

        if(os.path.exists("/content")):
            script = "/content/tugas-akhir/train/train.py"
            dataset_repo_or_path = "/content/encoded_data"
            !mkdir "/output"
            output_dir = "/output"
            config_name = "/content/tugas-akhir/train/config"

        elif(os.path.exists("/notebooks")):
            script = "/notebooks/github/train/train.py"
            dataset_repo_or_path = "/notebooks/github/train/datasets/encoded_data"
            !mkdir "/output"
            output_dir = "/output"
            config_name = "/notebooks/github/train/config"

        elif(os.path.exists("/kaggle")):
            script = "/kaggle/working/tugas-akhir/train/train.py"
            dataset_repo_or_path = "/kaggle/input/celeba-hq-encoded-256-bs60-sf60"
            !mkdir "/kaggle/working/tugas-akhir/train/output"
            output_dir = "/kaggle/working/tugas-akhir/train/output"
            config_name = "/kaggle/working/tugas-akhir/train/config"

        else:
            print("Where are you?")

        args = f"{script} --dataset_repo_or_path {dataset_repo_or_path} --output_dir {output_dir} --config_name {config_name} --tokenizer_name {tokenizer_name} --per_device_train_batch_size {per_device_train_batch_size} --num_train_epochs {num_train_epochs} --mp_devices {mp_devices} --dtype {dtype} --assert_tpu_available {assert_tpu_available} --save_steps {save_steps}"

    case 2:
        if(os.path.exists("/content")):
            args = "/content/tugas-akhir/app/app.py"

        elif(os.path.exists("/notebooks")):
            args = "/notebooks/github/app/app.py"

        elif(os.path.exists("/kaggle")):
            args = "/kaggle/working/tugas-akhir/app/app.py"

        else:
            print("Where are you?")

    case _:
        print("What are you going to do? Please check the usage.")

get_ipython().system(f"python {args}")